# Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import sys

#To import modules
sys.path.append('../')
from notebooks.dataset_helper import DatasetHelper
from codetector.src.features.shared.data.models.tokenizers import NLTKTokenizer, TikTokenTokenizer
from codetector import CodeSampleModel

# Remove Later

In [2]:
from codetector.src.features.shared.data.models.dataset.parquet_dataset import ParquetDataset
class TestParquetDataset(ParquetDataset):
    def getContentType(self):
        return CodeSampleModel

    def preProcess(self):
        pass

    def getTag(self):
        return 'test_parquet'

# Scatter plot code

## Heuristic

In [3]:
def heuristic(x:float,y:float) -> float:
    """
    Likelihood heuristic that the token contains valuable information.
    """
    #abs
    #Signed: Positive = Human, Negative = AI
    distFromIden = (y-x)/np.sqrt(2)
    distFromOrigin = np.sqrt(x**2 + y**2)
    return distFromIden * distFromOrigin

## Plot

In [10]:
def scatterPlot(positives:dict[str,int], negatives:dict[str,int], generatorTag:str=None, printHeuristic:bool=False, folderPath:str=None) -> None:
    #True negatives
    xs = []
    #True positives
    ys = []

    plt.xlabel("Human")
    plt.ylabel(generatorTag)

    #https://stackoverflow.com/questions/7908636/how-to-add-hovering-annotations-to-a-plot

    xSum = 0
    ySum = 0
    tokens = []

    for key in positives:
        ys.append(positives[key])
        tokens.append(key)
        ySum += positives[key]
        if key in negatives:
            xs.append(negatives[key])
            xSum += negatives[key]
        else:
            xs.append(0) 


    for key in negatives:
        if not(key in positives):
            tokens.append(key)
            xs.append(negatives[key])
            xSum += negatives[key]
            ys.append(0)

    #Skip if empty
    if xSum == 0 or ySum == 0:
        return
    
    #Normalize values by token count.
    xs = list(map(lambda x: x/xSum, xs))
    ys = list(map(lambda x: x/ySum, ys))

    colours = 'black'
    if printHeuristic:
        topTokens = [item for item in sorted(map(lambda x: (heuristic(x[0],x[1]),x[2]), zip(xs,ys,tokens)),key=lambda x : abs(x[0]), reverse=True)][:15]
        topTokens = list(map(lambda x: (x[1], 'human' if x[0]<0 else 'ai',x[0]), topTokens))
        # print(topTokens)
        #Convert to list of tokens
        topTokens = list(map(lambda x: x[0], topTokens))
        colours = ['red' if val in topTokens else 'black' for val in tokens]

        #https://stackoverflow.com/questions/14432557/scatter-plot-with-different-text-at-each-data-point
        for x,y,token in zip(xs,ys,tokens):
            if token in topTokens:
                i = topTokens.index(token)
                #https://stackoverflow.com/questions/18707338/print-raw-string-from-variable-not-getting-the-answers
                plt.annotate(f"{repr(token)}: {i}",(x,y),fontsize=8)

    plt.scatter(xs,ys,s=10,c=colours)

    maxVal = max(max(xs),max(ys))

    #Scale axes
    plt.xlim(0,maxVal)
    plt.ylim(0,maxVal)


    #Identity
    plt.plot([0,maxVal],[0,maxVal],c='red')


    if generatorTag:
        plt.title(f'Human vs {generatorTag}')
        plt.tight_layout()

    # plt.show()
    # plt.savefig(f'figures/freq/{generatorTag}.png')
    if folderPath:
        plt.rcParams['svg.fonttype'] = 'none'
        plt.savefig(f'{folderPath}/{generatorTag}.png',dpi=100)
        plt.savefig(f'{folderPath}/{generatorTag}.svg')
    else:
        plt.show()
    plt.close()

# Main

In [5]:
parq = TestParquetDataset('../data/generated_parquet')
parq.loadDataset()

print('Loaded dataset')

df = parq.toDataframe()
print('Converted to dataframe')

helper = DatasetHelper()

generators = list(df['Generator'].value_counts().keys())
generators.remove('human')

Loaded dataset
Converted to dataframe


In [13]:
tokenizer = TikTokenTokenizer()
# tokenizer = NLTKTokenizer()

## All Datasets

In [14]:
humanTokenFreq = helper.generateTokenFrequencies(df.loc[df['Generator'] == 'human'],tokenizer)
for generator in generators:
    genTokenFreq = helper.generateTokenFrequencies(df.loc[(df['Generator'] == generator) &
                                                   ((df['TopP']== 0.95) & (df['Temperature'] == 0.97))],
                                                   tokenizer)
    scatterPlot(genTokenFreq, humanTokenFreq, generator,printHeuristic=True,folderPath='./figures/token_frequencies/tiktoken/all_datasets')


Counting tokens: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 67.36it/s]


## By Dataset

In [15]:
datasets = ['stackoverflow-post','stackoverflow-pre','hf_apps','hf_codesearchnet-python','hf_leetcode-pre', 'leetcode-post']
for dataset in datasets:
    humanTokenFreq = helper.generateTokenFrequencies(df.loc[(df['Dataset'] == dataset)&(df['Generator'] == 'human')],tokenizer)
    for generator in generators:
        genTokenFreq = helper.generateTokenFrequencies(df.loc[(df['Dataset'] == dataset)&(df['Generator'] == generator) &
                                                       ((df['TopP']== 0.95) & (df['Temperature'] == 0.97))],
                                                       tokenizer)
        scatterPlot(genTokenFreq, humanTokenFreq, generator,printHeuristic=True,folderPath=f'./figures/token_frequencies/tiktoken/by_dataset/{dataset}')


Counting tokens: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 16.25it/s]
Counting tokens: 0it [00:00, ?it/s]
Counting tokens: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 58.56it/s]
Counting tokens: 0it [00:00, ?it/s]
Counting tokens: 0it [00:00, ?it/s]
Counting tokens: 0it [00:00, ?it/s]
Counting tokens: 0it [00:00, ?it/s]
Counting tokens: 0it [00:00, ?it/s]
Counting tokens: 0it [00:00, ?it/s]
Counting tokens: 0it [00:00, ?it/s]
Counting tokens: 0it [00:00, ?it/s]
Counting tokens: 0it [00:00, ?it/s]
Counting tokens: 0it [00:00, ?it/s]
Counting tokens: 0it [00:00, ?it/s]
Counting tokens: 0it [00:00, ?it/s]
Counting tokens: 0it [00:00, ?it/s]
Counting tokens: 0it [00:00, ?it/s]
Counting tokens: 0it [00:00, ?it/s]
Counting tokens: 0it [00:00, ?it/s]
Counting tokens: 0it [00:00, ?it/s]
Counting tokens: 0it [00:00, ?it/s

## Python Only

In [16]:

humanTokenFreq = helper.generateTokenFrequencies(df.loc[(df['Language'] == 'python')&(df['Generator'] == 'human')],tokenizer)
for generator in generators:
    genTokenFreq = helper.generateTokenFrequencies(df.loc[(df['Language'] == 'python')&(df['Generator'] == generator) &
                                                   ((df['TopP']== 0.95) & (df['Temperature'] == 0.97))],
                                                   tokenizer)
    scatterPlot(genTokenFreq, humanTokenFreq, generator,printHeuristic=True,folderPath='./figures/token_frequencies/tiktoken/python_only')


Counting tokens: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 64.96it/s]
